Este notebook tiene como objetivo realizar un análisis exploratorio de los datos de métricas recolectados durante la Fase 1 del proyecto, con el fin de identificar patrones, anomalías y señales relevantes asociadas a la inyección de fallos en los microservicios simulados.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8')

ModuleNotFoundError: No module named 'pandas'